In [1]:
from pyspark.sql import functions as F
from service.utils.spark import get_spark_session
spark = get_spark_session(dev=True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/23 06:22:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
test_namespace = 'gold' 
# spark.sql(f"DROP NAMESPACE IF EXISTS {test_namespace} CASCADE")
spark.sql(f"CREATE NAMESPACE IF NOT EXISTS {test_namespace}").show()

++
||
++
++



In [3]:
copmlete_order_timestamp = spark.read.table('warehousedev.gold.complete_order_timestamp')
copmlete_order_timestamp.show(n=5)

+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+
|            order_id|         customer_id|           purchase|           approved|  delivered_carrier| delivered_customer|
+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+
|a9a93c428c6103f21...|ef026b181b8edf96c...|2017-01-14 17:57:50|2017-01-17 10:55:13|2017-01-20 17:58:58|2017-01-30 18:37:44|
|56ef80c564f6fd57c...|6bd4f06d5599d085d...|2017-01-16 14:24:22|2017-01-16 14:35:17|2017-01-16 15:21:56|2017-01-23 08:56:05|
|f9427374480e37251...|71f9d647cd3876fd5...|2017-01-17 14:57:45|2017-01-18 02:10:16|2017-01-19 09:12:51|2017-01-24 15:14:01|
|d6d7c431275f0029d...|3855608848a57e1e3...|2017-01-19 14:28:48|2017-01-19 14:41:56|2017-01-24 10:05:23|2017-01-31 12:26:08|
|0957ed870116e596b...|903fccb1294b588a6...|2017-01-29 22:14:49|2017-01-29 22:33:34|2017-01-30 08:27:47|2017-02-08 17:14:55|
+-------

In [4]:
order_item_df = spark.read.csv("s3a://warehousedev/bronze/tsv/order_item.tsv", header=True, sep='\t')
product_df = spark.read.csv("s3a://warehousedev/bronze/tsv/product.tsv", header=True, sep='\t')

taget_category = 'health_beauty'
taget_category_product = product_df.filter(product_df.category == taget_category).select(['product_id', 'category'])
taget_category_order = taget_category_product.join(order_item_df, on='product_id', how='inner')
taget_category_order = taget_category_order.sort('product_id')

# 하나의 order에 여러 item이 있을 수 있으므로, order_item_id는 남겨둔다.
# `total_price` 는 상품가와 배송비를 합산한 가격
taget_category_order = taget_category_order.withColumn('total_price', F.round(F.col('price') + F.col('freight_value'), 4))
taget_category_order = taget_category_order.drop('shipping_limit_date', 'price', 'freight_value')
taget_category_order.show(n=5)


25/09/23 06:23:04 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+--------------------+-------------+--------------------+-------------+--------------------+-----------+
|          product_id|     category|            order_id|order_item_id|           seller_id|total_price|
+--------------------+-------------+--------------------+-------------+--------------------+-----------+
|00210e41887c2a8ef...|health_beauty|2d8e71bf7d31a41d4...|            1|e9779976487b77c6d...|      48.08|
|00210e41887c2a8ef...|health_beauty|1fd6c29ecb9dd8b65...|            1|e9779976487b77c6d...|      41.25|
|00210e41887c2a8ef...|health_beauty|226975521c585d7bb...|            1|e9779976487b77c6d...|      49.09|
|00210e41887c2a8ef...|health_beauty|2d8e71bf7d31a41d4...|            2|e9779976487b77c6d...|      48.08|
|00210e41887c2a8ef...|health_beauty|c01f1e6ceafe26dc7...|            1|e9779976487b77c6d...|      48.08|
+--------------------+-------------+--------------------+-------------+--------------------+-----------+
only showing top 5 rows



In [5]:
complete_target_category_order_df = taget_category_order.join(copmlete_order_timestamp, on='order_id', how='inner')
complete_target_category_order_df.show(n=5)

+--------------------+--------------------+-------------+-------------+--------------------+-----------+--------------------+-------------------+-------------------+-------------------+-------------------+
|            order_id|          product_id|     category|order_item_id|           seller_id|total_price|         customer_id|           purchase|           approved|  delivered_carrier| delivered_customer|
+--------------------+--------------------+-------------+-------------+--------------------+-----------+--------------------+-------------------+-------------------+-------------------+-------------------+
|0005a1a1728c9d785...|310ae3c140ff94b03...|health_beauty|            1|a416b6a846a117243...|      157.6|639d23421f5517f69...|2018-03-19 18:40:33|2018-03-20 18:35:21|2018-03-28 00:37:42|2018-03-29 18:17:31|
|00061f2a7bc09da83...|d63c1011f49d98b97...|health_beauty|            1|cc419e0650a3c5ba7...|      68.87|107e6259485efac66...|2018-03-24 22:16:10|2018-03-24 22:28:09|2018-03-27 

In [6]:
# gold
sale_stats = complete_target_category_order_df.groupBy('product_id').agg(
    F.count('order_id').alias('order_count'),
    F.round(F.sum(F.col('total_price')), 4).alias('total_sales')
    ).orderBy(F.col('order_count').desc())

sale_stats = sale_stats.withColumn('mean_sale', F.round(F.col('total_sales') / F.col('order_count'), 4))
sale_stats.show()

+--------------------+-----------+-----------+---------+
|          product_id|order_count|total_sales|mean_sale|
+--------------------+-----------+-----------+---------+
|154e7e31ebfa09220...|        274|     9824.2|  35.8547|
|2b4609f8948be1887...|        255|   26127.92| 102.4624|
|7c1bd920dbdf22470...|        220|    16735.7|  76.0714|
|bb50f2e236e5eea01...|        194|   67258.03| 346.6909|
|19c91ef95d509ea33...|        154|   24038.01|  156.091|
|6cdd53843498f9289...|        153|   57933.73| 378.6518|
|437c05a395e9e47f9...|        151|    9335.88|   61.827|
|3fbc0ef745950c793...|        149|   13680.15|  91.8131|
|e0cf79767c5b01625...|        137|    5484.97|  40.0363|
|4c2394abfbac7ff59...|        120|   12205.38| 101.7115|
|8c292ca193d326152...|         89|   15201.61| 170.8046|
|7fb04722aba7a2b63...|         87|    8963.46| 103.0283|
|634256dbcd184b375...|         72|    2532.05|  35.1674|
|2948658cb6abc8284...|         68|    9609.22| 141.3121|
|24c66f106f642621e...|         

In [7]:
# 1. 기준점(Threshold) 계산
# percentile_approx 함수를 사용하여 분위수 계산
order_count_threshold = sale_stats.agg(
    F.expr("percentile_approx(order_count, 0.75)")
).collect()[0][0]

median_avg_price = sale_stats.agg(
    F.expr("percentile_approx(mean_sale, 0.5)")
).collect()[0][0]

# print(order_count_threshold)
# print(median_avg_price)


# 2. 'group' 컬럼 추가
# when/otherwise와 col 함수를 사용하여 조건에 따라 그룹을 분류합니다.
target_product_sales_stats_segment = sale_stats.withColumn("segment",
    F.when((F.col("order_count") >= order_count_threshold) & (F.col("mean_sale") >= median_avg_price), "Star Products")
    .when((F.col("order_count") >= order_count_threshold) & (F.col("mean_sale") < median_avg_price), "Volume Drivers")
    .when((F.col("order_count") < order_count_threshold) & (F.col("mean_sale") >= median_avg_price), "Niche Gems")
    .otherwise("Question Marks")
)

target_product_sales_stats_segment.show(n=5, truncate=False)
target_product_sales_stats_segment.writeTo(f"{test_namespace}.target_product_sales_stats_segment").createOrReplace()

+--------------------------------+-----------+-----------+---------+--------------+
|product_id                      |order_count|total_sales|mean_sale|segment       |
+--------------------------------+-----------+-----------+---------+--------------+
|154e7e31ebfa092203795c972e5804a6|274        |9824.2     |35.8547  |Volume Drivers|
|2b4609f8948be18874494203496bc318|255        |26127.92   |102.4624 |Star Products |
|7c1bd920dbdf22470b68bde975dd3ccf|220        |16735.7    |76.0714  |Volume Drivers|
|bb50f2e236e5eea0100680137654686c|194        |67258.03   |346.6909 |Star Products |
|19c91ef95d509ea33eda93495c4d3481|154        |24038.01   |156.091  |Star Products |
+--------------------------------+-----------+-----------+---------+--------------+
only showing top 5 rows



# 구매기록에 Segment join
- Segment type: `Start Product`, `Volume Drivers`, `Niche Gems`, `Question Marks`

In [8]:
complete_target_category_order_df.show(n=4)

+--------------------+--------------------+-------------+-------------+--------------------+-----------+--------------------+-------------------+-------------------+-------------------+-------------------+
|            order_id|          product_id|     category|order_item_id|           seller_id|total_price|         customer_id|           purchase|           approved|  delivered_carrier| delivered_customer|
+--------------------+--------------------+-------------+-------------+--------------------+-----------+--------------------+-------------------+-------------------+-------------------+-------------------+
|0005a1a1728c9d785...|310ae3c140ff94b03...|health_beauty|            1|a416b6a846a117243...|      157.6|639d23421f5517f69...|2018-03-19 18:40:33|2018-03-20 18:35:21|2018-03-28 00:37:42|2018-03-29 18:17:31|
|00061f2a7bc09da83...|d63c1011f49d98b97...|health_beauty|            1|cc419e0650a3c5ba7...|      68.87|107e6259485efac66...|2018-03-24 22:16:10|2018-03-24 22:28:09|2018-03-27 

In [9]:
target_product_order_info_with_segment = complete_target_category_order_df.join(target_product_sales_stats_segment.select('product_id', 'segment'), on='product_id', how='inner')
target_product_order_info_with_segment.show(n=5)

+--------------------+--------------------+-------------+-------------+--------------------+-----------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------+
|          product_id|            order_id|     category|order_item_id|           seller_id|total_price|         customer_id|           purchase|           approved|  delivered_carrier| delivered_customer|      segment|
+--------------------+--------------------+-------------+-------------+--------------------+-----------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------+
|05f0fe07929d35be0...|362cd36939c1c4638...|health_beauty|            1|c70c1b0d8ca86052f...|     118.35|32ca12567c820aa0b...|2018-06-25 23:00:42|2018-06-27 08:53:50|2018-06-29 14:16:00|2018-07-02 21:05:36|Star Products|
|05f0fe07929d35be0...|b41cc5a0872a7fd88...|health_beauty|            1|c70c1b0d8ca86052f...|     118.35|ada8dea5e28ec3e9

In [10]:
target_product_order_info_with_segment.writeTo(f"{test_namespace}.target_product_order_info_with_segment").createOrReplace()

In [11]:
spark.stop()